In [6]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import os
import plotly.io as pio
pio.templates.default = "plotly_white"
import pandas as pd
from pathlib import Path
import tqdm
from tqdm.autonotebook import tqdm
import missingno as msno
from itertools import cycle
from sklearn.metrics import mean_absolute_error
from IPython.display import display, HTML
# %load_ext autoreload
# %autoreload 2
np.random.seed()
tqdm.pandas()

C:\Users\MRM\AppData\Local\Temp\ipykernel_18976\2316701429.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [28]:
folder = Path(r'C:\Users\MRM\Desktop\Data_Analytics\Medium_and_PPB\Machine_Learning\Machine_Learning\Time_Series_Forecasting\Modern-Time-Series-Forecasting-with-Python-main\data\london_smart_meters\hhblock_dataset\hhblock_dataset')
dummyfile = 'block_0.csv'
sample = Path(folder, dummyfile)
block_1 = pd.read_csv(sample, parse_dates=False)

block_1['day'] = pd.to_datetime(block_1['day'], yearfirst=True)

block_1.head()
                     


,LCLid,day,hh_0,hh_1,hh_2,hh_3,hh_4,hh_5,hh_6,hh_7,...,hh_38,hh_39,hh_40,hh_41,hh_42,hh_43,hh_44,hh_45,hh_46,hh_47
0,MAC000002,2012-10-13,0.263,0.269,0.275,0.256,0.211,0.136,0.161,0.119,...,0.918,0.278,0.267,0.239,0.230,0.233,0.235,0.188,0.259,0.250
1,MAC000002,2012-10-14,0.262,0.166,0.226,0.088,0.126,0.082,0.123,0.083,...,1.075,0.956,0.821,0.745,0.712,0.511,0.231,0.210,0.278,0.159
2,MAC000002,2012-10-15,0.192,0.097,0.141,0.083,0.132,0.070,0.130,0.074,...,1.164,0.249,0.225,0.258,0.260,0.334,0.299,0.236,0.241,0.237
3,MAC000002,2012-10-16,0.237,0.237,0.193,0.118,0.098,0.107,0.094,0.109,...,0.966,0.172,0.192,0.228,0.203,0.211,0.188,0.213,0.157,0.202
4,MAC000002,2012-10-17,0.157,0.211,0.155,0.169,0.101,0.117,0.084,0.118,...,0.223,0.075,0.230,0.208,0.265,0.377,0.327,0.277,0.288,0.256


CONVERT TO TS COMPACT FORM

In [29]:
block_1.columns.tolist()[2:]

['hh_0',
 'hh_1',
 'hh_2',
 'hh_3',
 'hh_4',
 'hh_5',
 'hh_6',
 'hh_7',
 'hh_8',
 'hh_9',
 'hh_10',
 'hh_11',
 'hh_12',
 'hh_13',
 'hh_14',
 'hh_15',
 'hh_16',
 'hh_17',
 'hh_18',
 'hh_19',
 'hh_20',
 'hh_21',
 'hh_22',
 'hh_23',
 'hh_24',
 'hh_25',
 'hh_26',
 'hh_27',
 'hh_28',
 'hh_29',
 'hh_30',
 'hh_31',
 'hh_32',
 'hh_33',
 'hh_34',
 'hh_35',
 'hh_36',
 'hh_37',
 'hh_38',
 'hh_39',
 'hh_40',
 'hh_41',
 'hh_42',
 'hh_43',
 'hh_44',
 'hh_45',
 'hh_46',
 'hh_47']

In [30]:
b2 = block_1.melt(id_vars=['LCLid','day'], value_vars=block_1.columns.tolist()[2:]).rename(columns={"variable": "hour_block", 'value': "energy_consumption"})
b2.head()

,LCLid,day,hour_block,energy_consumption
0,MAC000002,2012-10-13,hh_0,0.263
1,MAC000002,2012-10-14,hh_0,0.262
2,MAC000002,2012-10-15,hh_0,0.192
3,MAC000002,2012-10-16,hh_0,0.237
4,MAC000002,2012-10-17,hh_0,0.157


In [39]:
b2['offset'] = b2['hour_block'].str.replace("hh_", "").astype(int)
b2.sort_values(['day',"offset"], inplace=True)
b2.head()

,LCLid,day,hour_block,energy_consumption,offset
498,MAC000246,2011-12-04,hh_0,0.175,0
25784,MAC000246,2011-12-04,hh_1,0.098,1
51070,MAC000246,2011-12-04,hh_2,0.144,2
76356,MAC000246,2011-12-04,hh_3,0.065,3
101642,MAC000246,2011-12-04,hh_4,0.071,4


In [62]:
b2.energy_consumption.values

array([0.175, 0.098, 0.144, ..., 0.073, 0.258, 0.192])

In [32]:
start_date = b2['day'].min()
start_date

Timestamp('2011-12-04 00:00:00')

In [65]:
def add_energy(df):
    return df.energy_consumption.values.tolist()


In [77]:
b3 = b2.groupby(['LCLid']).apply(add_energy)
b3

LCLid
MAC000002    [0.263, 0.2689999999999999, 0.275, 0.256, 0.21...
MAC000246    [0.175, 0.098, 0.144, 0.065, 0.071, 0.037, 0.0...
MAC000450    [1.337, 1.426, 0.996, 0.971, 0.994, 0.952, 0.8...
MAC001074    [0.18, 0.086, 0.106, 0.173, 0.146, 0.223, 0.21...
MAC003223    [0.076, 0.079, 0.123, 0.109, 0.051, 0.069, 0.0...
MAC003239    [0.062, 0.081, 0.062, 0.063, 0.078, 0.064, 0.0...
MAC003252    [0.131, 0.213, 0.201, 0.166, 0.123, 0.149, 0.1...
MAC003281    [0.059, 0.04, 0.042, 0.064, 0.024, 0.073, 0.02...
MAC003305    [0.098, 0.096, 0.094, 0.129, 0.122, 0.139, 0.1...
MAC003348    [0.083, 0.054, 0.035, 0.073, 0.040999999999999...
MAC003388    [0.214, 0.2, 0.152, 0.086, 0.165, 0.1669999999...
MAC003394    [0.19, 0.176, 0.188, 0.182, 0.1689999999999999...
MAC003400    [0.093, 0.078, 0.052, 0.05, 0.05, 0.052, 0.081...
MAC003422    [0.183, 0.184, 0.207, 0.184, 0.132, 0.05799999...
MAC003423    [0.186, 0.172, 0.176, 0.186, 0.163999999999999...
MAC003428    [0.0559999999999999, 0.069, 0.071, 0

In [73]:
start_date = b2['day'].min()
max_date = b2['day'].max()
name = b2['LCLid'].unique()[0]
    ### Fill missing dates with NaN ###
    # Create a date range from  min to max
dr = pd.date_range(start=b2['day'].min(), end=max_date, freq="1D")
    # Add hh_0 to hh_47 to columns and with some unstack magic recreating date-hh_x combinations
dr = pd.DataFrame(columns=[f"hh_{i}" for i in range(48)], index=dr).unstack().reset_index()
    # renaming the columns
dr.columns = ["hour_block", "day", "_"]
    # left merging the dataframe to the standard dataframe
    # now the missing values will be left as NaN

dr = dr.merge(b2, on=['hour_block','day'], how='left')
    # sorting the rows
dr.sort_values(['day',"offset"], inplace=True)
    # extracting the timeseries array
ts = dr['energy_consumption'].values
len_ts = len(ts)

dr.head()

,hour_block,day,_,LCLid,energy_consumption,offset
0,hh_0,2011-12-04,NaN,MAC000246,0.175,0
25286,hh_1,2011-12-04,NaN,MAC000246,0.098,1
50572,hh_2,2011-12-04,NaN,MAC000246,0.144,2
75858,hh_3,2011-12-04,NaN,MAC000246,0.065,3
101144,hh_4,2011-12-04,NaN,MAC000246,0.071,4
